In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Cargar la base de datos
file_path = "nueva_base_+300_final.csv"  # Asegúrate de colocar la ruta correcta
df = pd.read_csv(file_path)

# Corregir el nombre de la columna de secuencia si es necesario
if "Seqence" in df.columns:
    df.rename(columns={"Seqence": "Sequence"}, inplace=True)

# Mapear la actividad a valores numéricos
df['Activity'] = df['Activity'].map({'Antibacterial': 1, 'Otros': 0})

# Extraer caracteres únicos y crear un diccionario de índices
all_amino_acids = set("".join(df['Sequence']))
amino_acids_dict = {aa: i for i, aa in enumerate(sorted(all_amino_acids))}

# Función para codificar secuencias con one-hot encoding
def one_hot_encode_sequence(sequence, max_length, char_dict):
    encoded = np.zeros((max_length, len(char_dict)), dtype=np.float32)
    for i, char in enumerate(sequence[:max_length]):
        if char in char_dict:
            encoded[i, char_dict[char]] = 1.0
    return encoded

# Ajustar las secuencias al mismo tamaño
max_length = max(df['Sequence'].apply(len))
X = np.array([one_hot_encode_sequence(seq, max_length, amino_acids_dict) for seq in df['Sequence']])
y = df['Activity'].values

# Aplanar las matrices para SVM (convertir cada secuencia en un solo vector)
X = X.reshape(X.shape[0], -1)

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar el modelo SVM
svm_model = SVC(kernel="linear")
svm_model.fit(X_train, y_train)

# Evaluar el modelo
y_pred = svm_model.predict(X_test)
report = classification_report(y_test, y_pred, target_names=["Otros", "Antibacterial"])

print(report)


               precision    recall  f1-score   support

        Otros       0.66      0.69      0.67       102
Antibacterial       0.89      0.88      0.89       298

     accuracy                           0.83       400
    macro avg       0.78      0.78      0.78       400
 weighted avg       0.83      0.83      0.83       400



In [2]:
# Lista de los aminoácidos presentes en tu diccionario
valid_amino_acids = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

# Filtrar secuencias que contengan caracteres no esperados
invalid_sequences = df[df['Sequence'].str.contains(f'[^{valid_amino_acids}]', regex=True)]

# Mostrar las filas con secuencias que contienen caracteres no válidos
print(invalid_sequences)


            Title                  Sequence  Length  Activity
299  DAL-PEG-LK6   YaGFLRXIKKILSKIKKLLK-NH2    24.0         1
